In [ ]:
# @title
!pip install -q yt_dlp

import os
import json
import base64
import subprocess
import sys
from IPython.display import display, HTML
from google.colab import output
import yt_dlp

try:
    import yt_dlp
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "yt_dlp"])
    import yt_dlp

def get_library():
    files = sorted([f for f in os.listdir('.') if f.endswith('.mp3')])
    data = {}
    for f_name in files:
        try:
            with open(f_name, "rb") as f:
                enc = base64.b64encode(f.read()).decode('utf-8')
                data[f_name] = f"data:audio/mpeg;base64,{enc}"
        except: pass
    return files, data

def dl_handler(query):
    output.eval_js("showStatus('Searching SoundCloud...', true)")

    if query.startswith("http") or query.startswith("www"):
        search_query = query
    else:
        search_query = f"scsearch1:{query}"

    opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{'key': 'FFmpegExtractAudio','preferredcodec': 'mp3','preferredquality': '192'}],
        'outtmpl': '%(title)s.%(ext)s',
        'quiet': True,
        'noplaylist': True,
        'default_search': 'scsearch1',
        'extractor_args': {'youtube': {'player_client': ['android', 'web']}}
    }

    try:
        with yt_dlp.YoutubeDL(opts) as ydl:
            ydl.download([search_query])
        f, d = get_library()
        output.eval_js(f'updateLib({json.dumps(f)}, {json.dumps(d)})')
        output.eval_js("showStatus('Import Complete', false)")
    except Exception as e:
        output.eval_js(f"showStatus('Error: Check Console', false)")
        print(e)

def del_handler(fname):
    if os.path.exists(fname):
        os.remove(fname)
        f, d = get_library()
        output.eval_js(f'updateLib({json.dumps(f)}, {json.dumps(d)}, "{fname}")')

output.register_callback('k8o5.dl', dl_handler)
output.register_callback('k8o5.del', del_handler)

files, data = get_library()

HTML_CODE = f"""
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<title>k8o5/studio</title>
<style>
@import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700;800&display=swap');

:root {{
    --bg: #000000;
    --surf: #0a0a0a;
    --border: #1a1a1a;
    --text: #ffffff;
    --acc: #fff;
}}

* {{ box-sizing: border-box; margin: 0; padding: 0; outline: none; user-select: none; -webkit-user-drag: none; }}
body {{ background: var(--bg); color: var(--text); font-family: 'Inter', sans-serif; height: 100vh; width: 100vw; overflow: hidden; }}

#app {{
    width: 100%; height: 100%;
    display: grid;
    grid-template-rows: 60px 1fr 90px;
    grid-template-columns: 240px 1fr;
    grid-template-areas: "head head" "side main" "play play";
    background: var(--bg);
}}

.header {{
    grid-area: head; border-bottom: 1px solid var(--border);
    display: flex; align-items: center; justify-content: space-between;
    padding: 0 24px; background: rgba(0,0,0,0.8); backdrop-filter: blur(10px); z-index: 50;
}}
.brand {{ font-weight: 800; font-size: 18px; color: #fff; display:flex; align-items:center; gap:8px; }}
.brand span {{ opacity: 0.5; font-weight: 400; }}

.status-bar {{
    background: #111; border: 1px solid #222; padding: 6px 14px; border-radius: 20px;
    font-size: 12px; font-weight: 500; display: flex; align-items: center; gap: 8px;
    opacity: 0; transition: opacity 0.3s;
}}
.status-bar.show {{ opacity: 1; }}
.spinner {{ width: 12px; height: 12px; border: 2px solid #333; border-top-color: #fff; border-radius: 50%; animation: spin 0.8s linear infinite; display: none; }}
@keyframes spin {{ to {{ transform: rotate(360deg); }} }}

.sidebar {{
    grid-area: side; background: var(--surf); border-right: 1px solid var(--border);
    padding: 20px 12px; display: flex; flex-direction: column; gap: 4px;
}}
.nav-btn {{
    padding: 12px 16px; border-radius: 8px; cursor: pointer; color: #666;
    font-size: 14px; font-weight: 600; display: flex; align-items: center; gap: 12px; transition: 0.2s;
}}
.nav-btn:hover {{ color: #fff; background: rgba(255,255,255,0.03); }}
.nav-btn.active {{ color: #fff; background: #161616; }}
.nav-btn svg {{ width: 20px; height: 20px; }}

.main {{ grid-area: main; position: relative; overflow: hidden; background: #000; }}
.page {{ display: none; padding: 32px; height: 100%; overflow-y: auto; position: relative; z-index: 2; }}
.page.active {{ display: block; }}

h1 {{ font-size: 28px; font-weight: 700; margin-bottom: 24px; letter-spacing: -0.5px; }}

.grid {{ display: grid; grid-template-columns: repeat(auto-fill, minmax(160px, 1fr)); gap: 24px; padding-bottom: 100px; }}
.card {{ cursor: pointer; group: 1; }}
.art-box {{
    width: 100%; aspect-ratio: 1; border-radius: 12px; background: #1a1a1a;
    position: relative; overflow: hidden; margin-bottom: 12px;
    box-shadow: 0 4px 15px rgba(0,0,0,0.5); transition: transform 0.2s;
    border: 1px solid rgba(255,255,255,0.05);
}}
.card:hover .art-box {{ transform: translateY(-4px); border-color: rgba(255,255,255,0.2); }}
.art-grad {{ width: 100%; height: 100%; display: flex; align-items: center; justify-content: center; color: rgba(255,255,255,0.1); }}
.play-overlay {{
    position: absolute; inset: 0; background: rgba(0,0,0,0.3);
    display: flex; align-items: center; justify-content: center; opacity: 0; transition: 0.2s;
}}
.card:hover .play-overlay {{ opacity: 1; }}
.play-circle {{ width: 40px; height: 40px; background: rgba(255,255,255,0.9); border-radius: 50%; display: flex; align-items: center; justify-content: center; color: #000; box-shadow: 0 4px 10px rgba(0,0,0,0.5); transform: scale(0.9); transition: 0.2s; }}
.card:hover .play-circle {{ transform: scale(1); }}

.song-title {{ font-size: 13px; font-weight: 600; color: #fff; overflow: hidden; white-space: nowrap; text-overflow: ellipsis; }}
.song-sub {{ font-size: 12px; color: #555; margin-top: 2px; }}

.del-btn {{ position: absolute; top: 8px; right: 8px; width: 24px; height: 24px; background: rgba(0,0,0,0.6); border-radius: 50%; color: #fff; display: flex; align-items: center; justify-content: center; opacity: 0; transition: 0.2s; z-index: 10; }}
.card:hover .del-btn {{ opacity: 1; }}
.del-btn:hover {{ background: #ff3333; }}

/* FX RACK STYLES */
.fx-grid {{ display: grid; grid-template-columns: repeat(auto-fit, minmax(240px, 1fr)); gap: 24px; }}
.fx-card {{ background: #0f0f0f; border: 1px solid #222; padding: 24px; border-radius: 12px; }}
.fx-title {{ font-size: 12px; font-weight: 700; color: #666; text-transform: uppercase; margin-bottom: 20px; letter-spacing: 1px; }}
.slider-group {{ margin-bottom: 24px; }}
.slider-info {{ display: flex; justify-content: space-between; margin-bottom: 10px; font-size: 13px; font-weight: 500; }}
.range {{ -webkit-appearance: none; width: 100%; height: 4px; background: #222; border-radius: 2px; outline: none; }}
.range::-webkit-slider-thumb {{ -webkit-appearance: none; width: 16px; height: 16px; background: #fff; border-radius: 50%; cursor: pointer; border: 2px solid #000; }}

/* IMPORT STYLES */
.search-box {{
    max-width: 500px; margin: 0 auto; display: flex; flex-direction: column;
    align-items: center; justify-content: center; height: 70%; text-align: center;
}}
.inp {{
    width: 100%; background: #111; border: 1px solid #333; color: #fff;
    padding: 16px 20px; border-radius: 12px; font-size: 16px; margin-bottom: 20px; transition: 0.2s;
}}
.inp:focus {{ border-color: #666; background: #161616; }}
.btn-dl {{
    background: #fff; color: #000; border: none; padding: 12px 32px;
    border-radius: 24px; font-weight: 700; font-size: 14px; cursor: pointer; transition: 0.2s;
}}
.btn-dl:hover {{ transform: scale(1.05); }}

.player {{
    grid-area: play; background: rgba(10,10,10,0.95); border-top: 1px solid var(--border);
    display: grid; grid-template-columns: 240px 1fr 240px; align-items: center; padding: 0 24px; z-index: 50;
}}

.meta {{ display: flex; align-items: center; gap: 14px; }}
.meta-img {{ width: 48px; height: 48px; background: #222; border-radius: 6px; display: flex; align-items: center; justify-content: center; overflow:hidden; }}
.meta-info {{ display: flex; flex-direction: column; overflow: hidden; }}
.meta-name {{ font-size: 13px; font-weight: 600; color: #fff; white-space: nowrap; overflow: hidden; text-overflow: ellipsis; max-width: 160px; }}
.meta-stat {{ font-size: 11px; color: #555; }}

.ctrl {{ display: flex; flex-direction: column; align-items: center; gap: 6px; width: 100%; }}
.ctrl-btns {{ display: flex; align-items: center; gap: 24px; }}
.c-btn {{ background: none; border: none; color: #aaa; cursor: pointer; transition: 0.2s; }}
.c-btn:hover {{ color: #fff; }}
.play-btn {{ width: 32px; height: 32px; background: #fff; border-radius: 50%; color: #000; display: flex; align-items: center; justify-content: center; transition: 0.2s; }}
.play-btn:hover {{ transform: scale(1.1); }}

.prog-bar {{ width: 100%; max-width: 500px; height: 4px; background: #222; border-radius: 2px; position: relative; cursor: pointer; }}
.prog-fill {{ width: 0%; height: 100%; background: #fff; border-radius: 2px; position: relative; }}
.prog-handle {{ position: absolute; right: -5px; top: -3px; width: 10px; height: 10px; background: #fff; border-radius: 50%; opacity: 0; transition: 0.1s; }}
.prog-bar:hover .prog-handle {{ opacity: 1; }}

.vol-wrap {{ display: flex; justify-content: flex-end; align-items: center; gap: 12px; }}
.vol-bar {{ width: 80px; height: 4px; background: #222; border-radius: 2px; position: relative; cursor: pointer; }}
.vol-fill {{ width: 70%; height: 100%; background: #aaa; border-radius: 2px; }}
.vol-bar:hover .vol-fill {{ background: #fff; }}

#viz {{ position: absolute; top: 0; left: 0; width: 100%; height: 100%; pointer-events: none; opacity: 0.3; mix-blend-mode: screen; z-index: 1; }}
.fs-btn {{ position: absolute; top: 18px; right: 24px; background: none; border: 1px solid #333; color: #888; padding: 4px 10px; border-radius: 4px; cursor: pointer; font-size: 11px; }}
.fs-btn:hover {{ border-color: #666; color: #fff; }}

</style>
</head>
<body>

<div id="app">
    <div class="header">
        <div class="brand">k8o5<span>/studio</span></div>
        <div class="status-bar" id="status"><div class="spinner"></div><span id="stat-txt">Ready</span></div>
        <button class="fs-btn" onclick="toggleFS()">FULLSCREEN</button>
    </div>

    <div class="sidebar">
        <div class="nav-btn active" onclick="nav('lib', this)">
            <svg viewBox="0 0 24 24" fill="currentColor"><path d="M4 6H2v14c0 1.1.9 2 2 2h14v-2H4V6zm16-4H8c-1.1 0-2 .9-2 2v12c0 1.1.9 2 2 2h12c1.1 0 2-.9 2-2V4c0-1.1-.9-2-2-2zm0 14H8V4h12v12z"/></svg> Library
        </div>
        <div class="nav-btn" onclick="nav('fx', this)">
            <svg viewBox="0 0 24 24" fill="currentColor"><path d="M3 17v3a2 2 0 002 2h14a2 2 0 002-2v-3a2 2 0 00-2-2H5a2 2 0 00-2 2zM3 6v3a2 2 0 002 2h14a2 2 0 002-2V6a2 2 0 00-2-2H5a2 2 0 00-2 2z"/></svg> FX Rack
        </div>
        <div class="nav-btn" onclick="nav('dl', this)">
            <svg viewBox="0 0 24 24" fill="currentColor"><path d="M19 9h-4V3H9v6H5l7 7 7-7zM5 18v2h14v-2H5z"/></svg> Import
        </div>
    </div>

    <div class="main">
        <canvas id="viz"></canvas>

        <!-- LIBRARY -->
        <div id="p-lib" class="page active">
            <h1>Library</h1>
            <div id="grid" class="grid"></div>
        </div>

        <!-- FX RACK -->
        <div id="p-fx" class="page">
            <h1>Audio Effects</h1>
            <div class="fx-grid">
                <div class="fx-card">
                    <div class="fx-title">Playback</div>
                    <div class="slider-group">
                        <div class="slider-info"><span>Speed</span><span id="v-spd">1.0x</span></div>
                        <input type="range" class="range" id="sl-spd" min="0.5" max="2.0" step="0.01" value="1">
                    </div>
                </div>
                <div class="fx-card">
                    <div class="fx-title">Distortion & Drive</div>
                    <div class="slider-group">
                        <div class="slider-info"><span>Distortion</span><span id="v-dst">0%</span></div>
                        <input type="range" class="range" id="sl-dst" min="0" max="1" step="0.01" value="0">
                    </div>
                </div>
                <div class="fx-card">
                    <div class="fx-title">Atmosphere</div>
                    <div class="slider-group">
                        <div class="slider-info"><span>Reverb</span><span id="v-rev">0%</span></div>
                        <input type="range" class="range" id="sl-rev" min="0" max="1" step="0.01" value="0">
                    </div>
                </div>
                <div class="fx-card">
                    <div class="fx-title">Equalizer</div>
                    <div class="slider-group">
                        <div class="slider-info"><span>Lows</span><span id="v-low">0 dB</span></div>
                        <input type="range" class="range" id="sl-low" min="-20" max="20" step="1" value="0">
                    </div>
                    <div class="slider-group">
                        <div class="slider-info"><span>Highs</span><span id="v-hi">0 dB</span></div>
                        <input type="range" class="range" id="sl-hi" min="-20" max="20" step="1" value="0">
                    </div>
                </div>
            </div>
        </div>

        <!-- DOWNLOADER -->
        <div id="p-dl" class="page">
            <div class="search-box">
                <svg viewBox="0 0 24 24" width="64" height="64" fill="#333" style="margin-bottom:20px"><path d="M12 2C6.48 2 2 6.48 2 12s4.48 10 10 10 10-4.48 10-10S17.52 2 12 2zm-1 17.93c-3.95-.49-7-3.85-7-7.93 0-.62.08-1.21.21-1.79L9 15v1c0 1.1.9 2 2 2v1.93zm6.9-2.54c-.26-.81-1-1.39-1.9-1.39h-1v-3c0-.55-.45-1-1-1H8v-2h2c.55 0 1-.45 1-1V7h2c1.1 0 2-.9 2-2v-.41c2.93 1.19 5 4.06 5 7.41 0 2.08-.8 3.97-2.1 5.39z"/></svg>
                <h2 style="margin-bottom:10px; font-weight:600">SoundCloud Import</h2>
                <p style="color:#666; margin-bottom:20px; font-size:13px">Paste a SoundCloud URL or search.</p>
                <input type="text" id="dl-in" class="inp" placeholder="Search or URL...">
                <button class="btn-dl" onclick="dl()">IMPORT TRACK</button>
            </div>
        </div>
    </div>

    <div class="player">
        <div class="meta">
            <div class="meta-img" id="art"><svg width="24" height="24" fill="#333" viewBox="0 0 24 24"><path d="M12 3v9.28a4.39 4.39 0 00-1.5-.28C8.01 12 6 14.01 6 16.5S8.01 21 10.5 21c2.31 0 4.2-1.75 4.45-4H15V6h4V3h-7z"/></svg></div>
            <div class="meta-info"><div class="meta-name" id="t-name">No Track</div><div class="meta-stat" id="t-stat">Idle</div></div>
        </div>
        <div class="ctrl">
            <div class="ctrl-btns">
                <button class="c-btn" onclick="prev()"><svg width="20" height="20" fill="currentColor" viewBox="0 0 24 24"><path d="M6 6h2v12H6zm3.5 6l8.5 6V6z"/></svg></button>
                <button class="play-btn" id="bp" onclick="toggle()"><svg width="14" height="14" fill="currentColor" viewBox="0 0 24 24"><path d="M8 5v14l11-7z"/></svg></button>
                <button class="c-btn" onclick="next()"><svg width="20" height="20" fill="currentColor" viewBox="0 0 24 24"><path d="M6 18l8.5-6L6 6v12zM16 6v12h2V6h-2z"/></svg></button>
            </div>
            <div class="prog-bar" id="prog"><div class="prog-fill" id="pfill"><div class="prog-handle"></div></div></div>
        </div>
        <div class="vol-wrap">
            <svg width="18" height="18" fill="#888" viewBox="0 0 24 24"><path d="M3 9v6h4l5 5V4L7 9H3zm13.5 3c0-1.77-1.02-3.29-2.5-4.03v8.05c1.48-.73 2.5-2.25 2.5-4.02zM14 3.23v2.06c2.89.86 5 3.54 5 6.71s-2.11 5.85-5 6.71v2.06c4.01-.91 7-4.49 7-8.77s-2.99-7.86-7-8.77z"/></svg>
            <div class="vol-bar" id="vol"><div class="vol-fill" id="vfill"></div></div>
        </div>
    </div>
</div>

<script>
    let files = {json.dumps(files)};
    let data = {json.dumps(data)};
    let idx=-1, playing=false, ac, src, gain, anal, buf, st=0, ot=0, dur=0;

    // FX Nodes
    let low, high, dist, verb, dry, wet;

    const $ = id => document.getElementById(id);

    function init() {{
        if(ac) return;
        ac = new (window.AudioContext || window.webkitAudioContext)();

        // Nodes
        gain = ac.createGain();
        anal = ac.createAnalyser(); anal.fftSize = 256;
        low = ac.createBiquadFilter(); low.type="lowshelf"; low.frequency.value=320;
        high = ac.createBiquadFilter(); high.type="highshelf"; high.frequency.value=3200;
        dist = ac.createWaveShaper();
        verb = ac.createConvolver();
        dry = ac.createGain();
        wet = ac.createGain();

        // Impulse Response for Reverb
        let len = 2 * ac.sampleRate;
        let imp = ac.createBuffer(2, len, ac.sampleRate);
        for(let i=0; i<2; i++) {{
            let ch = imp.getChannelData(i);
            for(let j=0; j<len; j++) ch[j] = (Math.random()*2-1)*Math.pow(1-j/len, 2);
        }}
        verb.buffer = imp;

        // Route: Src -> Dist -> Low -> High -> Gain -> Anal -> Dry/Wet -> Dest
        gain.connect(anal);
        anal.connect(dry); dry.connect(ac.destination);
        anal.connect(wet); wet.connect(verb); verb.connect(ac.destination);
        gain.gain.value = 0.7;

        applyFx();
        vizLoop();
    }}

    async function load(i) {{
        init();
        if(playing) stop(true);
        idx = i;
        let n = files[i];
        $('t-name').innerText = n.replace('.mp3','');
        $('t-stat').innerText = "Loading...";
        let h = (n.charCodeAt(0)*10)%360;
        $('art').style.background = `linear-gradient(135deg, hsl(${{h}},50%,20%), hsl(${{h}},50%,10%))`;

        try {{
            let b = Uint8Array.from(atob(data[n].split(',')[1]), c => c.charCodeAt(0)).buffer;
            buf = await ac.decodeAudioData(b);
            dur = buf.duration;
            play(0);
        }} catch(e) {{ $('t-stat').innerText = "Error"; }}
    }}

    function play(off) {{
        src = ac.createBufferSource(); src.buffer = buf;
        // Connect Source to start of FX chain
        src.connect(dist); dist.connect(low); low.connect(high); high.connect(gain);

        applyFx(); // Apply current slider values
        src.start(0, off);
        st = ac.currentTime - off; ot = 0; playing = true;
        $('bp').innerHTML = `<svg width="14" height="14" fill="currentColor" viewBox="0 0 24 24"><path d="M6 19h4V5H6v14zm8-14v14h4V5h-4z"/></svg>`;
        $('t-stat').innerText = "Playing";
        src.onended = () => {{ if(playing && (ac.currentTime-st)>=dur-0.1) next(); }};
    }}

    function stop(mem) {{
        if(src) {{ src.stop(); src.disconnect(); }}
        if(playing) ot = ac.currentTime - st;
        playing = false; if(!mem) ot = 0;
        $('bp').innerHTML = `<svg width="14" height="14" fill="currentColor" viewBox="0 0 24 24"><path d="M8 5v14l11-7z"/></svg>`;
        $('t-stat').innerText = "Paused";
    }}

    function toggle() {{
        if(idx<0 && files.length) load(0);
        else if(playing) stop(true);
        else if(buf) play(ot);
    }}

    function next() {{ if(files.length) load((idx+1)%files.length); }}
    function prev() {{ if(files.length) load((idx-1+files.length)%files.length); }}

    // FX LOGIC
    function applyFx() {{
        if(!ac) return;
        let s_spd = $('sl-spd').value; $('v-spd').innerText = s_spd+'x';
        if(src) src.playbackRate.value = s_spd;

        let s_low = $('sl-low').value; $('v-low').innerText = s_low+' dB';
        low.gain.value = s_low;

        let s_hi = $('sl-hi').value; $('v-hi').innerText = s_hi+' dB';
        high.gain.value = s_hi;

        let s_rev = $('sl-rev').value; $('v-rev').innerText = Math.round(s_rev*100)+'%';
        dry.gain.value = 1 - s_rev; wet.gain.value = s_rev;

        let s_dst = $('sl-dst').value; $('v-dst').innerText = Math.round(s_dst*100)+'%';
        if(s_dst > 0) {{
            let k = s_dst * 100, n = 44100, curve = new Float32Array(n), deg = Math.PI / 180;
            for(let i=0; i<n; ++i) {{
                let x = i * 2 / n - 1;
                curve[i] = (3+k)*x*20*deg/(Math.PI+k*Math.abs(x));
            }}
            dist.curve = curve;
        }} else dist.curve = null;
    }}
    ['sl-spd','sl-low','sl-hi','sl-rev','sl-dst'].forEach(id => $(id).addEventListener('input', applyFx));


    // UI & EVENTS
    $('prog').addEventListener('click', e => {{
        if(!buf) return;
        let r = $('prog').getBoundingClientRect();
        stop(true); play(dur * ((e.clientX - r.left)/r.width));
    }});

    function loop() {{
        if(playing) $('pfill').style.width = ((ac.currentTime-st)/dur)*100 + '%';
        requestAnimationFrame(loop);
    }}
    loop();

    let dragVol = false;
    const setVol = (e) => {{
        if(!gain) return;
        let r = $('vol').getBoundingClientRect();
        let p = Math.max(0, Math.min(1, (e.clientX - r.left)/r.width));
        $('vfill').style.width = (p*100)+'%';
        gain.gain.value = p;
    }};
    $('vol').addEventListener('mousedown', (e) => {{ dragVol = true; setVol(e); }});
    window.addEventListener('mousemove', (e) => {{ if(dragVol) setVol(e); }});
    window.addEventListener('mouseup', () => dragVol = false);

    const cv = $('viz'), cx = cv.getContext('2d');
    const res = () => {{ cv.width = window.innerWidth; cv.height = window.innerHeight; }};
    window.onresize = res; res();
    function vizLoop() {{
        requestAnimationFrame(vizLoop);
        if(!anal) return;
        let d = new Uint8Array(anal.frequencyBinCount);
        anal.getByteFrequencyData(d);
        cx.clearRect(0,0,cv.width,cv.height);
        let bw = (cv.width/d.length)*2.5, x=0;
        for(let i=0; i<d.length; i++) {{
            let h = (d[i]/255)*400;
            cx.fillStyle = `rgba(255,255,255,${{d[i]/800}})`;
            cx.fillRect(x, cv.height-h, bw, h);
            x+=bw+1;
        }}
    }}

    function render() {{
        let h = '';
        if(!files.length) h = `<div style="grid-column:1/-1;text-align:center;padding:50px;color:#333">Library Empty</div>`;
        files.forEach((f, i) => {{
            let c = (f.charCodeAt(0)*20)%360;
            h+=`<div class="card" onclick="load(${{i}})">
                <div class="art-box">
                    <div class="art-grad" style="background:linear-gradient(45deg, hsl(${{c}},40%,20%), hsl(${{c}},40%,10%))">
                        <svg width="48" height="48" fill="currentColor" viewBox="0 0 24 24"><path d="M12 3v9.28a4.39 4.39 0 00-1.5-.28C8.01 12 6 14.01 6 16.5S8.01 21 10.5 21c2.31 0 4.2-1.75 4.45-4H15V6h4V3h-7z"/></svg>
                    </div>
                    <div class="play-overlay"><div class="play-circle"><svg width="24" height="24" viewBox="0 0 24 24" fill="currentColor"><path d="M8 5v14l11-7z"/></svg></div></div>
                    <div class="del-btn" onclick="del('${{f}}', event)"><svg width="14" height="14" viewBox="0 0 24 24" fill="currentColor"><path d="M6 19c0 1.1.9 2 2 2h8c1.1 0 2-.9 2-2V7H6v12zM19 4h-3.5l-1-1h-5l-1 1H5v2h14V4z"/></svg></div>
                </div>
                <div class="song-title">${{f.replace('.mp3','')}}</div>
                <div class="song-sub">SoundCloud Import</div>
            </div>`;
        }});
        $('grid').innerHTML = h;
    }}

    window.nav = (p, el) => {{
        document.querySelectorAll('.page').forEach(x=>x.classList.remove('active'));
        $('p-'+p).classList.add('active');
        document.querySelectorAll('.nav-btn').forEach(x=>x.classList.remove('active'));
        el.classList.add('active');
    }};

    window.dl = () => {{
        let v = $('dl-in').value;
        if(v) google.colab.kernel.invokeFunction('k8o5.dl', [v]);
    }};
    window.del = (n, e) => {{
        e.stopPropagation();
        if(confirm('Delete?')) google.colab.kernel.invokeFunction('k8o5.del', [n]);
    }};
    window.toggleFS = () => {{
        if(!document.fullscreenElement) document.documentElement.requestFullscreen();
        else if(document.exitFullscreen) document.exitFullscreen();
    }};

    window.updateLib = (f, d, rm) => {{ files=f; data=d; if(rm && idx>-1) stop(); render(); }};
    window.showStatus = (m, l) => {{
        $('status').classList.add('show'); $('stat-txt').innerText=m;
        document.querySelector('.spinner').style.display=l?'block':'none';
        if(!l) setTimeout(()=>$('status').classList.remove('show'),3000);
    }};

    render();
</script>
</body>
</html>
"""
display(HTML(HTML_CODE))